## 1️⃣ Importación de Librerías

In [15]:
# Manipulación de datos
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Estadísticas y series temporales
from scipy import stats
from scipy.stats import pearsonr, spearmanr
from statsmodels.tsa.stattools import grangercausalitytests, ccf, acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


---

## 2️⃣ Carga de Datos

In [ ]:
# Configurar rutas
BASE_DIR = Path('/home/els4nchez/Videos/TECH')
DATA_DIR = BASE_DIR / 'data'
UNIFICACION_DIR = BASE_DIR / 'unificacion'
DATA_PROCESADO_DIR = UNIFICACION_DIR / 'datos_procesados'
FIGURAS_DIR = UNIFICACION_DIR / 'figuras'

# Crear directorio de figuras si no existe
FIGURAS_DIR.mkdir(parents=True, exist_ok=True)

print("📂 Cargando datasets...\n")

# 1. Cargar precios con outliers (del notebook 04)
ruta_precios = DATA_PROCESADO_DIR / 'precios_oro_con_outliers.csv'
df_precios = pd.read_csv(ruta_precios, index_col=0, parse_dates=True)
print(f"✅ Precios cargados: {df_precios.shape[0]:,} días")

# 2. Cargar sentimientos diarios (del notebook 05)
ruta_sentimientos = DATA_PROCESADO_DIR / 'sentimientos_diarios.csv'
df_sentimientos = pd.read_csv(ruta_sentimientos, index_col=0, parse_dates=True)
print(f"✅ Sentimientos cargados: {df_sentimientos.shape[0]:,} días")

# 3. Cargar noticias individuales (opcional, para análisis detallado)
ruta_noticias = DATA_PROCESADO_DIR / 'noticias_oro_con_sentimientos.csv'
df_noticias = pd.read_csv(ruta_noticias, parse_dates=['fecha'])
print(f"✅ Noticias cargadas: {df_noticias.shape[0]:,} noticias")

print(f"\n📅 Rango temporal:")
print(f"   Precios: {df_precios.index.min().strftime('%Y-%m-%d')} → {df_precios.index.max().strftime('%Y-%m-%d')}")
print(f"   Sentimientos: {df_sentimientos.index.min().strftime('%Y-%m-%d')} → {df_sentimientos.index.max().strftime('%Y-%m-%d')}")

📂 Cargando datasets...

✅ Precios cargados: 3,614 días
✅ Sentimientos cargados: 2,273 días
✅ Noticias cargadas: 18,776 noticias

📅 Rango temporal:
   Precios: 2016-01-03 → 2025-11-24
   Sentimientos: 2016-01-03 → 2025-10-31


---

## 3️⃣ Integración de Datos (Merge por Fecha)

In [17]:
# Merge de precios y sentimientos por fecha
df_merged = df_precios.join(df_sentimientos, how='inner', rsuffix='_sent')

# Renombrar columnas para claridad
df_merged = df_merged.rename(columns={
    'sentiment_weighted': 'sentiment',
    'sentiment_ma_7': 'sentiment_ma7',
    'sentiment_ma_30': 'sentiment_ma30'
})

print(f"📊 Dataset integrado creado")
print(f"   Total de días con datos completos: {len(df_merged):,}")
print(f"   Rango: {df_merged.index.min().strftime('%Y-%m-%d')} → {df_merged.index.max().strftime('%Y-%m-%d')}")
print(f"\n🔍 Variables disponibles:")
print(f"   Precios: {[col for col in df_merged.columns if col in ['Open', 'High', 'Low', 'Close', 'Returns', 'Volatility_30']]}")
print(f"   Outliers: {[col for col in df_merged.columns if 'outlier' in col.lower()]}")
print(f"   Sentimientos: {[col for col in df_merged.columns if 'sentiment' in col.lower()]}")
print(f"\n📈 Primeras 3 filas:")
df_merged.head(3)

📊 Dataset integrado creado
   Total de días con datos completos: 2,273
   Rango: 2016-01-03 → 2025-10-31

🔍 Variables disponibles:
   Precios: ['Open', 'High', 'Low', 'Close', 'Returns', 'Volatility_30']
   Outliers: ['outlier_iqr', 'outlier_zscore', 'outlier_iforest', 'outlier_count', 'outlier_consensus']
   Sentimientos: ['sentiment_numeric', 'sentiment', 'sentiment_score', 'sentiment_ma7', 'sentiment_ma30', 'sentiment_extreme']

📈 Primeras 3 filas:


,Open,High,Low,Close,Volume,Returns,Volatility_30,outlier_iqr,z_score,outlier_zscore,...,outlier_iforest,outlier_count,outlier_consensus,sentiment_numeric,sentiment,sentiment_score,num_noticias,sentiment_ma7,sentiment_ma30,sentiment_extreme
2016-01-03,1062.341,1066.321,1061.729,1063.059,1.61136,NaN,NaN,False,1.195449,False,...,True,1,False,0.083333,0.036220,0.813113,12,NaN,NaN,False
2016-01-04,1063.059,1083.488,1062.239,1072.979,76.56875,0.009288,NaN,False,1.179487,False,...,True,1,False,0.153846,0.104399,0.835821,13,NaN,NaN,False
2016-01-05,1072.990,1082.048,1072.629,1077.632,72.43959,0.004327,NaN,False,1.172000,False,...,True,1,False,-0.263158,-0.226055,0.776322,19,NaN,NaN,False


---

## 4️⃣ Análisis de Correlación

In [18]:
# Calcular correlaciones entre sentimiento y variables de precios
print("📊 MATRIZ DE CORRELACIÓN: SENTIMIENTOS vs PRECIOS\n")
print("="*80)

variables_precio = ['Close', 'Returns', 'Volatility_30']
variables_sentimiento = ['sentiment', 'sentiment_ma7', 'sentiment_ma30']

correlaciones = {}

for var_sent in variables_sentimiento:
    print(f"\n{var_sent.upper()}:")
    print("-" * 70)
    
    for var_precio in variables_precio:
        # Filtrar datos válidos (sin NaN)
        datos_validos = df_merged[[var_sent, var_precio]].dropna()
        
        if len(datos_validos) > 0:
            # Pearson (lineal)
            pearson_r, pearson_p = pearsonr(datos_validos[var_sent], datos_validos[var_precio])
            
            # Spearman (monotónica, no lineal)
            spearman_r, spearman_p = spearmanr(datos_validos[var_sent], datos_validos[var_precio])
            
            correlaciones[f"{var_sent}_vs_{var_precio}"] = {
                'pearson_r': pearson_r,
                'pearson_p': pearson_p,
                'spearman_r': spearman_r,
                'spearman_p': spearman_p
            }
            
            print(f"  {var_precio:20s} │ Pearson: {pearson_r:+.4f} (p={pearson_p:.4f}) │ Spearman: {spearman_r:+.4f} (p={spearman_p:.4f})")

print("\n" + "="*80)
print("\n💡 Interpretación:")
print("   - Correlación > 0: Sentimiento positivo asociado con aumentos de precio")
print("   - Correlación < 0: Sentimiento positivo asociado con caídas de precio")
print("   - |Correlación| < 0.3: Correlación débil")
print("   - 0.3 ≤ |Correlación| < 0.7: Correlación moderada")
print("   - |Correlación| ≥ 0.7: Correlación fuerte")
print("   - p-value < 0.05: Correlación estadísticamente significativa")

📊 MATRIZ DE CORRELACIÓN: SENTIMIENTOS vs PRECIOS


SENTIMENT:
----------------------------------------------------------------------
  Close                │ Pearson: -0.0612 (p=0.0035) │ Spearman: -0.0564 (p=0.0072)
  Returns              │ Pearson: +0.0057 (p=0.7857) │ Spearman: +0.0064 (p=0.7604)
  Volatility_30        │ Pearson: -0.0251 (p=0.2343) │ Spearman: -0.0280 (p=0.1839)

SENTIMENT_MA7:
----------------------------------------------------------------------
  Close                │ Pearson: -0.1559 (p=0.0000) │ Spearman: -0.1595 (p=0.0000)
  Returns              │ Pearson: +0.0036 (p=0.8655) │ Spearman: +0.0096 (p=0.6493)
  Volatility_30        │ Pearson: -0.0665 (p=0.0016) │ Spearman: -0.0476 (p=0.0239)

SENTIMENT_MA30:
----------------------------------------------------------------------
  Close                │ Pearson: -0.2688 (p=0.0000) │ Spearman: -0.3015 (p=0.0000)
  Returns              │ Pearson: -0.0152 (p=0.4706) │ Spearman: -0.0134 (p=0.5246)
  Volatility_30     

In [19]:
# Heatmap de correlaciones
# Crear matriz de correlación
vars_analisis = variables_precio + variables_sentimiento
corr_matrix = df_merged[vars_analisis].corr()

# Plotly heatmap
fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.index,
    colorscale='RdBu',
    zmid=0,
    text=corr_matrix.values,
    texttemplate='%{text:.3f}',
    textfont={"size": 10},
    colorbar=dict(title="Correlación")
))

fig.update_layout(
    title='🔥 Matriz de Correlación: Precios vs Sentimientos',
    xaxis_title='Variables',
    yaxis_title='Variables',
    height=600,
    template='plotly_white'
)

fig.show()

---

## 5️⃣ Visualización Temporal: Precios vs Sentimientos

In [20]:
# Gráfico de doble eje: Precio de cierre y sentimiento
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    subplot_titles=('Precio de Cierre del Oro (XAU/USD)', 'Sentimiento de Noticias (Ponderado)'),
    row_heights=[0.6, 0.4]
)

# Subplot 1: Precio de cierre
fig.add_trace(go.Scatter(
    x=df_merged.index,
    y=df_merged['Close'],
    mode='lines',
    name='Precio Cierre',
    line=dict(color='gold', width=1.5)
), row=1, col=1)

# Subplot 2: Sentimiento
fig.add_trace(go.Scatter(
    x=df_merged.index,
    y=df_merged['sentiment'],
    mode='lines',
    name='Sentimiento Diario',
    line=dict(color='lightblue', width=1),
    opacity=0.5
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=df_merged.index,
    y=df_merged['sentiment_ma7'],
    mode='lines',
    name='Sentimiento MA(7)',
    line=dict(color='blue', width=2)
), row=2, col=1)

# Línea de referencia en sentimiento 0
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

fig.update_xaxes(title_text="Fecha", row=2, col=1)
fig.update_yaxes(title_text="Precio (USD)", row=1, col=1)
fig.update_yaxes(title_text="Sentimiento", row=2, col=1)

fig.update_layout(
    title_text="📈 Evolución Temporal: Precio del Oro y Sentimiento de Noticias",
    height=700,
    template='plotly_white',
    hovermode='x unified',
    showlegend=True
)

fig.show()

---

## 6️⃣ Análisis de Eventos: Outliers vs Sentimientos Extremos

In [21]:
# Identificar días con outliers (consensus)
df_merged['has_outlier'] = df_merged['outlier_consensus'].astype(bool)

# Identificar días con sentimiento extremo (±1.5 std)
sentiment_mean = df_merged['sentiment'].mean()
sentiment_std = df_merged['sentiment'].std()
threshold_pos = sentiment_mean + 1.5 * sentiment_std
threshold_neg = sentiment_mean - 1.5 * sentiment_std

df_merged['sentiment_extreme'] = (
    (df_merged['sentiment'] > threshold_pos) | 
    (df_merged['sentiment'] < threshold_neg)
)

# Coincidencias: Días con AMBOS eventos
df_merged['outlier_AND_extreme_sentiment'] = (
    df_merged['has_outlier'] & df_merged['sentiment_extreme']
)

print("🔍 ANÁLISIS DE EVENTOS COINCIDENTES")
print("="*80)
print(f"\nTotal de días analizados: {len(df_merged):,}")
print(f"\n📊 Eventos individuales:")
print(f"   Días con outliers en precios: {df_merged['has_outlier'].sum():,} ({df_merged['has_outlier'].mean()*100:.2f}%)")
print(f"   Días con sentimiento extremo: {df_merged['sentiment_extreme'].sum():,} ({df_merged['sentiment_extreme'].mean()*100:.2f}%)")
print(f"\n🎯 Coincidencias:")
print(f"   Días con AMBOS eventos: {df_merged['outlier_AND_extreme_sentiment'].sum():,} ({df_merged['outlier_AND_extreme_sentiment'].mean()*100:.2f}%)")

# Tabla de contingencia (2x2)
contingency = pd.crosstab(
    df_merged['has_outlier'],
    df_merged['sentiment_extreme'],
    margins=True
)

print(f"\n📋 Tabla de Contingencia:")
print(contingency)

# Chi-cuadrado test
from scipy.stats import chi2_contingency
chi2, p_value, dof, expected = chi2_contingency(contingency.iloc[:-1, :-1])

print(f"\n🧪 Test Chi-Cuadrado de Independencia:")
print(f"   χ² = {chi2:.4f}")
print(f"   p-value = {p_value:.4f}")
print(f"   {'✅ SIGNIFICATIVO' if p_value < 0.05 else '❌ NO SIGNIFICATIVO'} (α=0.05)")

if p_value < 0.05:
    print(f"\n   💡 Los outliers de precios y sentimientos extremos NO son independientes.")
    print(f"      Existe una asociación estadísticamente significativa.")
else:
    print(f"\n   💡 Los outliers de precios y sentimientos extremos son independientes.")
    print(f"      No hay evidencia de asociación significativa.")

🔍 ANÁLISIS DE EVENTOS COINCIDENTES

Total de días analizados: 2,273

📊 Eventos individuales:
   Días con outliers en precios: 98 (4.31%)
   Días con sentimiento extremo: 269 (11.83%)

🎯 Coincidencias:
   Días con AMBOS eventos: 11 (0.48%)

📋 Tabla de Contingencia:
sentiment_extreme  False  True   All
has_outlier                         
False               1917   258  2175
True                  87    11    98
All                 2004   269  2273

🧪 Test Chi-Cuadrado de Independencia:
   χ² = 0.0010
   p-value = 0.9750
   ❌ NO SIGNIFICATIVO (α=0.05)

   💡 Los outliers de precios y sentimientos extremos son independientes.
      No hay evidencia de asociación significativa.


In [22]:
# Visualizar eventos coincidentes
fig = go.Figure()

# Precio de cierre
fig.add_trace(go.Scatter(
    x=df_merged.index,
    y=df_merged['Close'],
    mode='lines',
    name='Precio Cierre',
    line=dict(color='gray', width=1),
    opacity=0.5
))

# Marcar outliers de precio
outliers = df_merged[df_merged['has_outlier']]
fig.add_trace(go.Scatter(
    x=outliers.index,
    y=outliers['Close'],
    mode='markers',
    name='Outliers Precio',
    marker=dict(color='red', size=8, symbol='circle')
))

# Marcar coincidencias (outlier + sentimiento extremo)
coincidencias = df_merged[df_merged['outlier_AND_extreme_sentiment']]
fig.add_trace(go.Scatter(
    x=coincidencias.index,
    y=coincidencias['Close'],
    mode='markers',
    name='Outlier + Sentimiento Extremo',
    marker=dict(color='orange', size=12, symbol='star', line=dict(color='black', width=1))
))

fig.update_layout(
    title='🎯 Eventos Coincidentes: Outliers de Precio y Sentimientos Extremos',
    xaxis_title='Fecha',
    yaxis_title='Precio de Cierre (USD)',
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()

print(f"\n📌 Top 5 eventos coincidentes:")
if len(coincidencias) > 0:
    top_coincidencias = coincidencias.nlargest(5, 'Returns')[['Close', 'Returns', 'sentiment', 'num_noticias']]
    print(top_coincidencias)
else:
    print("   No se encontraron eventos coincidentes.")


📌 Top 5 eventos coincidentes:
               Close   Returns  sentiment  num_noticias
2025-10-16  4365.225  0.036377  -0.679530             5
2025-06-02  3387.198  0.023074  -0.618087             9
2025-05-29  3315.395  0.016988  -0.612884            11
2025-05-15  3236.845  0.015891  -0.749991             7
2025-09-08  3634.735  0.012162   0.343412             7


---

## 7️⃣ Análisis de Lag (Desfase Temporal)

In [23]:
# Calcular correlación cruzada (cross-correlation) con diferentes lags
print("🕐 ANÁLISIS DE LAG: ¿Cuántos días tarda el mercado en reaccionar a noticias?\n")
print("="*80)

max_lag = 10  # Analizar hasta 10 días de desfase

# Preparar datos sin NaN
datos_lag = df_merged[['sentiment', 'Returns']].dropna()

lag_correlations = []

for lag in range(-max_lag, max_lag + 1):
    if lag < 0:
        # Lag negativo: sentimiento DESPUÉS del precio (precio predice sentimiento)
        sent_shifted = datos_lag['sentiment'].shift(-lag)
    else:
        # Lag positivo: sentimiento ANTES del precio (sentimiento predice precio)
        sent_shifted = datos_lag['sentiment'].shift(lag)
    
    datos_validos = pd.DataFrame({
        'sentiment': sent_shifted,
        'Returns': datos_lag['Returns']
    }).dropna()
    
    if len(datos_validos) > 0:
        corr, p_value = pearsonr(datos_validos['sentiment'], datos_validos['Returns'])
        lag_correlations.append({
            'lag': lag,
            'correlation': corr,
            'p_value': p_value
        })

df_lags = pd.DataFrame(lag_correlations)

# Encontrar lag óptimo
best_lag = df_lags.loc[df_lags['correlation'].abs().idxmax()]

print(f"\n🎯 LAG ÓPTIMO:")
print(f"   Desfase: {int(best_lag['lag'])} días")
print(f"   Correlación: {best_lag['correlation']:.4f}")
print(f"   p-value: {best_lag['p_value']:.4f}")

if best_lag['lag'] > 0:
    print(f"\n   💡 El sentimiento de noticias PRECEDE los cambios de precio en ~{int(best_lag['lag'])} días.")
    print(f"      Evidencia de capacidad predictiva del sentimiento.")
elif best_lag['lag'] < 0:
    print(f"\n   💡 Los cambios de precio PRECEDEN los sentimientos en ~{abs(int(best_lag['lag']))} días.")
    print(f"      Las noticias reaccionan a movimientos del mercado.")
else:
    print(f"\n   💡 Correlación simultánea (lag=0). No hay desfase temporal.")

print(f"\n📊 Top 5 lags con mayor correlación absoluta:")
print(df_lags.nlargest(5, 'correlation', keep='all')[['lag', 'correlation', 'p_value']])

🕐 ANÁLISIS DE LAG: ¿Cuántos días tarda el mercado en reaccionar a noticias?


🎯 LAG ÓPTIMO:
   Desfase: -6 días
   Correlación: -0.0696
   p-value: 0.0009

   💡 Los cambios de precio PRECEDEN los sentimientos en ~6 días.
      Las noticias reaccionan a movimientos del mercado.

📊 Top 5 lags con mayor correlación absoluta:
    lag  correlation   p_value
5    -5     0.046612  0.026466
15    5     0.046612  0.026466
2    -8     0.018913  0.368402
18    8     0.018913  0.368402
9    -1     0.014426  0.492013
11    1     0.014426  0.492013


In [24]:
# Gráfico de correlaciones por lag
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_lags['lag'],
    y=df_lags['correlation'],
    marker_color=['red' if c < 0 else 'green' for c in df_lags['correlation']],
    text=df_lags['correlation'],
    texttemplate='%{text:.3f}',
    textposition='outside',
    name='Correlación'
))

# Marcar lag óptimo
fig.add_vline(
    x=best_lag['lag'],
    line_dash="dash",
    line_color="blue",
    annotation_text=f"Lag óptimo: {int(best_lag['lag'])} días",
    annotation_position="top"
)

fig.update_layout(
    title='📊 Correlación Cruzada: Sentimiento vs Retornos (por Lag)',
    xaxis_title='Lag (días)',
    yaxis_title='Correlación de Pearson',
    template='plotly_white',
    height=500
)

fig.add_hline(y=0, line_dash="dot", line_color="gray")

fig.show()

print("\n💡 Interpretación del gráfico:")
print("   - Lag positivo: Sentimiento ANTES del precio (predictivo)")
print("   - Lag negativo: Precio ANTES del sentimiento (reactivo)")
print("   - Barras verdes: Correlación positiva")
print("   - Barras rojas: Correlación negativa")


💡 Interpretación del gráfico:
   - Lag positivo: Sentimiento ANTES del precio (predictivo)
   - Lag negativo: Precio ANTES del sentimiento (reactivo)
   - Barras verdes: Correlación positiva
   - Barras rojas: Correlación negativa


---

## 8️⃣ Test de Causalidad de Granger

**Granger Causality** determina si una serie temporal puede predecir otra.

**Hipótesis Nula (H₀):** El sentimiento NO causa (en sentido de Granger) cambios en los retornos.

**Hipótesis Alternativa (H₁):** El sentimiento SÍ causa cambios en los retornos.

In [25]:
# Preparar datos para test de Granger
# Necesitamos series estacionarias
datos_granger = df_merged[['sentiment', 'Returns']].dropna()

print("🧪 TEST DE CAUSALIDAD DE GRANGER")
print("="*80)
print("\nHipótesis: ¿El sentimiento de noticias causa (en sentido de Granger) los retornos del oro?\n")

max_lag_granger = 5  # Probar hasta 5 días de lag

try:
    # Test de Granger: sentimiento → Returns
    print("📊 DIRECCIÓN: Sentimiento → Retornos")
    print("-" * 70)
    
    granger_result = grangercausalitytests(
        datos_granger[['Returns', 'sentiment']],  # [variable dependiente, variable independiente]
        maxlag=max_lag_granger,
        verbose=False
    )
    
    # Extraer p-values para cada lag
    granger_pvalues = []
    for lag in range(1, max_lag_granger + 1):
        # F-test p-value
        p_value = granger_result[lag][0]['ssr_ftest'][1]
        granger_pvalues.append({
            'lag': lag,
            'p_value': p_value,
            'significativo': 'SÍ' if p_value < 0.05 else 'NO'
        })
    
    df_granger = pd.DataFrame(granger_pvalues)
    print(df_granger.to_string(index=False))
    
    # Resultado general
    min_pvalue = df_granger['p_value'].min()
    best_lag_granger = df_granger.loc[df_granger['p_value'].idxmin(), 'lag']
    
    print(f"\n🎯 RESULTADO:")
    if min_pvalue < 0.05:
        print(f"   ✅ Se RECHAZA H₀ (p={min_pvalue:.4f} < 0.05)")
        print(f"   💡 El sentimiento SÍ causa (Granger) los retornos del oro.")
        print(f"   📌 Lag óptimo: {int(best_lag_granger)} días")
        print(f"\n   🔮 Implicación: El sentimiento de noticias tiene capacidad PREDICTIVA.")
    else:
        print(f"   ❌ NO se rechaza H₀ (p={min_pvalue:.4f} ≥ 0.05)")
        print(f"   💡 No hay evidencia de causalidad Granger.")
        print(f"\n   ⚠️  El sentimiento no predice significativamente los retornos.")

except Exception as e:
    print(f"❌ Error en test de Granger: {e}")
    print("   Posibles causas: datos insuficientes, series no estacionarias, etc.")

🧪 TEST DE CAUSALIDAD DE GRANGER

Hipótesis: ¿El sentimiento de noticias causa (en sentido de Granger) los retornos del oro?

📊 DIRECCIÓN: Sentimiento → Retornos
----------------------------------------------------------------------
 lag  p_value significativo
   1 0.487957            NO
   2 0.176249            NO
   3 0.302023            NO
   4 0.296015            NO
   5 0.086535            NO

🎯 RESULTADO:
   ❌ NO se rechaza H₀ (p=0.0865 ≥ 0.05)
   💡 No hay evidencia de causalidad Granger.

   ⚠️  El sentimiento no predice significativamente los retornos.
 lag  p_value significativo
   1 0.487957            NO
   2 0.176249            NO
   3 0.302023            NO
   4 0.296015            NO
   5 0.086535            NO

🎯 RESULTADO:
   ❌ NO se rechaza H₀ (p=0.0865 ≥ 0.05)
   💡 No hay evidencia de causalidad Granger.

   ⚠️  El sentimiento no predice significativamente los retornos.


In [26]:
# Test inverso: ¿Los retornos causan el sentimiento?
print("\n" + "="*80)
print("📊 DIRECCIÓN INVERSA: Retornos → Sentimiento")
print("-" * 70)

try:
    granger_result_inverse = grangercausalitytests(
        datos_granger[['sentiment', 'Returns']],  # Invertir orden
        maxlag=max_lag_granger,
        verbose=False
    )
    
    granger_pvalues_inv = []
    for lag in range(1, max_lag_granger + 1):
        p_value = granger_result_inverse[lag][0]['ssr_ftest'][1]
        granger_pvalues_inv.append({
            'lag': lag,
            'p_value': p_value,
            'significativo': 'SÍ' if p_value < 0.05 else 'NO'
        })
    
    df_granger_inv = pd.DataFrame(granger_pvalues_inv)
    print(df_granger_inv.to_string(index=False))
    
    min_pvalue_inv = df_granger_inv['p_value'].min()
    
    print(f"\n🎯 RESULTADO:")
    if min_pvalue_inv < 0.05:
        print(f"   ✅ Se RECHAZA H₀ (p={min_pvalue_inv:.4f} < 0.05)")
        print(f"   💡 Los retornos SÍ causan (Granger) el sentimiento.")
        print(f"\n   📰 Las noticias REACCIONAN a movimientos del mercado.")
    else:
        print(f"   ❌ NO se rechaza H₀ (p={min_pvalue_inv:.4f} ≥ 0.05)")
        print(f"   💡 No hay evidencia de causalidad inversa.")

except Exception as e:
    print(f"❌ Error en test inverso: {e}")


📊 DIRECCIÓN INVERSA: Retornos → Sentimiento
----------------------------------------------------------------------
 lag  p_value significativo
   1 0.449999            NO
   2 0.274389            NO
   3 0.201258            NO
   4 0.340298            NO
   5 0.451027            NO

🎯 RESULTADO:
   ❌ NO se rechaza H₀ (p=0.2013 ≥ 0.05)
   💡 No hay evidencia de causalidad inversa.


---

## 9️⃣ Exportación de Resultados

In [27]:
# Exportar dataset integrado
archivo_merged = DATA_PROCESADO_DIR / 'datos_integrados_precios_sentimientos.csv'
df_merged.to_csv(archivo_merged)
print(f"✅ Dataset integrado exportado: {archivo_merged}")
print(f"   Dimensiones: {df_merged.shape}")

# Exportar correlaciones
import json

resultados_correlacion = {
    'correlaciones_detalladas': correlaciones,
    'lag_analysis': {
        'lag_optimo': int(best_lag['lag']),
        'correlacion_optima': float(best_lag['correlation']),
        'p_value': float(best_lag['p_value']),
        'todas_correlaciones': df_lags.to_dict(orient='records')
    },
    'eventos_coincidentes': {
        'total_outliers': int(df_merged['has_outlier'].sum()),
        'total_sentimiento_extremo': int(df_merged['sentiment_extreme'].sum()),
        'total_coincidencias': int(df_merged['outlier_AND_extreme_sentiment'].sum()),
        'chi_cuadrado': float(chi2),
        'p_value_chi2': float(p_value)
    }
}

if 'df_granger' in locals():
    resultados_correlacion['granger_causality'] = {
        'sentimiento_a_retornos': df_granger.to_dict(orient='records'),
        'mejor_lag': int(best_lag_granger),
        'min_p_value': float(min_pvalue)
    }

if 'df_granger_inv' in locals():
    resultados_correlacion['granger_causality_inversa'] = {
        'retornos_a_sentimiento': df_granger_inv.to_dict(orient='records'),
        'min_p_value': float(min_pvalue_inv)
    }

archivo_resultados = DATA_PROCESADO_DIR / 'resultados_correlacion_causalidad.json'

with open(archivo_resultados, 'w', encoding='utf-8') as f:
    json.dump(resultados_correlacion, f, indent=2, ensure_ascii=False)

print(f"✅ Resultados de correlación y causalidad guardados: {archivo_resultados}")

✅ Dataset integrado exportado: /home/els4nchez/Videos/TECH/unificacion/datos_procesados/datos_integrados_precios_sentimientos.csv
   Dimensiones: (2273, 23)
✅ Resultados de correlación y causalidad guardados: /home/els4nchez/Videos/TECH/unificacion/datos_procesados/resultados_correlacion_causalidad.json


---

## 🔟 Resumen Ejecutivo

In [28]:
# Resumen ejecutivo
print("""
╔════════════════════════════════════════════════════════════════════════╗
║         🔗 RESUMEN EJECUTIVO - CORRELACIÓN Y CAUSALIDAD                ║
╠════════════════════════════════════════════════════════════════════════╣
║                                                                        ║
║  ✅ ANÁLISIS COMPLETADO EXITOSAMENTE                                   ║
║                                                                        ║
║  📊 DATOS INTEGRADOS:                                                  ║
║                                                                        ║""")

print(f"║  • Total de días analizados: {len(df_merged):,}                               ║")
print(f"║  • Rango: {df_merged.index.min().strftime('%Y-%m-%d')} → {df_merged.index.max().strftime('%Y-%m-%d')}                    ║")

print("""
║                                                                        ║
║  📈 CORRELACIONES PRINCIPALES:                                         ║
║                                                                        ║""")

# Mostrar correlaciones más fuertes
for key, val in list(correlaciones.items())[:3]:
    print(f"║  • {key:40s} r={val['pearson_r']:+.3f}           ║")

print(f"""
║                                                                        ║
║  🕐 ANÁLISIS DE LAG:                                                   ║
║                                                                        ║
║  • Lag óptimo: {int(best_lag['lag']):2d} días                                            ║
║  • Correlación en lag óptimo: {best_lag['correlation']:+.4f}                            ║
║                                                                        ║
║  🎯 EVENTOS COINCIDENTES:                                              ║
║                                                                        ║
║  • Outliers de precio: {df_merged['has_outlier'].sum():3d} días                                  ║
║  • Sentimientos extremos: {df_merged['sentiment_extreme'].sum():3d} días                              ║
║  • Coincidencias: {df_merged['outlier_AND_extreme_sentiment'].sum():3d} días                                      ║
║  • Chi² test: p={p_value:.4f} {'(significativo)' if p_value < 0.05 else '(no significativo)':20s}      ║
║                                                                        ║""")

if 'min_pvalue' in locals():
    print("""
║  🧪 CAUSALIDAD DE GRANGER:                                             ║
║                                                                        ║""")
    
    if min_pvalue < 0.05:
        print(f"║  • Sentimiento → Retornos: ✅ SIGNIFICATIVO (p={min_pvalue:.4f})          ║")
        print(f"║  • Lag óptimo: {int(best_lag_granger)} días                                              ║")
        print("║  • Conclusión: El sentimiento PREDICE los retornos                ║")
    else:
        print(f"║  • Sentimiento → Retornos: ❌ NO SIGNIFICATIVO (p={min_pvalue:.4f})       ║")
        print("║  • Conclusión: Sin evidencia de causalidad predictiva            ║")

print("""
║                                                                        ║
║  📂 ARCHIVOS GENERADOS:                                                ║
║                                                                        ║
║  • datos_integrados_precios_sentimientos.csv                          ║
║  • resultados_correlacion_causalidad.json                             ║
║                                                                        ║
║  ➡️  SIGUIENTE PASO:                                                   ║
║     Notebook 07 - Modelo LSTM con Features de Sentimiento             ║
║                                                                        ║
╚════════════════════════════════════════════════════════════════════════╝
""")


╔════════════════════════════════════════════════════════════════════════╗
║         🔗 RESUMEN EJECUTIVO - CORRELACIÓN Y CAUSALIDAD                ║
╠════════════════════════════════════════════════════════════════════════╣
║                                                                        ║
║  ✅ ANÁLISIS COMPLETADO EXITOSAMENTE                                   ║
║                                                                        ║
║  📊 DATOS INTEGRADOS:                                                  ║
║                                                                        ║
║  • Total de días analizados: 2,273                               ║
║  • Rango: 2016-01-03 → 2025-10-31                    ║

║                                                                        ║
║  📈 CORRELACIONES PRINCIPALES:                                         ║
║                                                                        ║
║  • sentiment_vs_Close                       r=-0.